In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/optimal_control_julia/lec_5/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using PyPlot

LoadError: InitError: Failed to import required Python module matplotlib.

For automated matplotlib installation, try configuring PyCall to use the Conda.jl package's Python "Miniconda" distribution within Julia. Relaunch Julia and run:
    ENV["PYTHON"]=""
    Pkg.build("PyCall")
before trying again.

Note that Conda.jl can use custom conda environment. Read more about `CONDA_JL_HOME` configuration in Conda.jl documentation:
    https://github.com/JuliaPy/Conda.jl

To install matplotlib using /home/kimsooyoung/.julia/conda/3/x86_64/bin/conda, you can run the following command from your system shell:
    /home/kimsooyoung/.julia/conda/3/x86_64/bin/conda install matplotlib

The pyimport exception was: PyError (PyImport_ImportModule

The Python package matplotlib could not be imported by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/home/kimsooyoung/.julia/conda/

In [3]:
Q = Diagonal([0.5; 1])
function f(x)
    return 0.5*(x-[1; 0])'*Q*(x-[1; 0])
end
function ∇f(x)
    return Q*(x-[1; 0])
end
function ∇2f(x)
    return Q
end

∇2f (generic function with 1 method)

In [4]:
function c(x)
    return x[1]^2 + 2*x[1] - x[2]
end
function ∂c(x)
    return [2*x[1]+2 -1]
end

∂c (generic function with 1 method)

In [5]:
function plot_landscape()
    Nsamp = 20
    Xsamp = kron(ones(Nsamp),LinRange(-4,4,Nsamp)')
    Ysamp = kron(ones(Nsamp)',LinRange(-4,4,Nsamp))
    Zsamp = zeros(Nsamp,Nsamp)
    for j = 1:Nsamp
        for k = 1:Nsamp
            Zsamp[j,k] = f([Xsamp[j,k]; Ysamp[j,k]])
        end
    end
    contour(Xsamp,Ysamp,Zsamp)

    xc = LinRange(-3.2,1.2,Nsamp)
    plot(xc,xc.^2+2.0.*xc,"y")
end
plot_landscape()

LoadError: UndefVarError: contour not defined

In [ ]:
function gauss_newton_step(x,λ)
    H = ∇2f(x)
    C = ∂c(x)
    Δz = [H C'; C 0]\[-∇f(x)-C'*λ; -c(x)]
    Δx = Δz[1:2]
    Δλ = Δz[3]
    return Δx, Δλ
end

In [ ]:
xguess = [-1; -1]
λguess = [0.0]
plot_landscape()
plot(xguess[1], xguess[2], "rx")

In [ ]:
Δx, Δλ = gauss_newton_step(xguess[:,end],λguess[end])
xguess = [xguess xguess[:,end]+Δx]
λguess = [λguess λguess[end]+Δλ]
plot_landscape()
plot(xguess[1,:], xguess[2,:], "rx")

In [ ]:
function P(x,λ)
    ∇L = [-∇f(x)-∂c(x)'*λ; -c(x)]
    return 0.5*dot(∇L,∇L)
end
function ∇P(x,λ)
    H = ∇2f(x) + ForwardDiff.jacobian(xn -> ∂c(xn)'*λ, x)
    C = ∂c(x)
    return [H C'; C 0]*[-∇f(x)-C'*λ; -c(x)]
end

In [ ]:
ρ = 1.0
function P(x,λ)
    f(x) + ρ*norm(c(x),1)
end
function ∇P(x,λ)
    [∇f(x) + ρ*∂c(x)'*sign.(c(x)); zeros(length(λ))]
end

In [ ]:
ρ = 1.0
function P(x,λ)
    f(x) + λ'*c(x) + 0.5*ρ*dot(c(x),c(x))
end
function ∇P(x,λ)
    [∇f(x) + ∂c(x)'*(λ + ρ*c(x)); c(x)]
end

In [ ]:
Δx, Δλ = gauss_newton_step(xguess[:,end],λguess[end])
α = 1
while P(xguess[:,end]+α*Δx, λguess[end]+α*Δλ) > P(xguess[:,end], λguess[end]) + 0.01*α*dot(∇P(xguess[:,end], λguess[end]),[Δx; Δλ])
    α = 0.5*α
end
xguess = [xguess xguess[:,end]+α*Δx]
λguess = [λguess λguess[end]+α*Δλ]
plot_landscape()
plot(xguess[1,:], xguess[2,:], "rx")
α